# Select the LCLS-II py3 kernel in the top right
# Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import psana as ps

# Specify experiment and run number. Then generate datasource

In [ ]:
exp = 'tmolv2918'
run_number = 215
ds = ps.DataSource(exp=exp, run=run_number)
run = next(ds.runs())

# Specify the detectors and analyses to conduct shot-by-shot and let the TMOanalysis library handle the rest

In [ ]:
detectors = {}
# Fast detectors
# detectors['sample']={'pskey':'timing', 'get':lambda det: det}
detectors['evrs'] = {'pskey':'timing', 'get':lambda det: det.raw.eventcodes}
detectors['tmo_atmopal']={'pskey':'tmo_atmopal', 'get':lambda det: det.raw.image}
detectors['vls']={'pskey':'andor', 'get':lambda det: det.raw.value}
detectors['gmd']={'pskey':'gmd', 'get':lambda det: det.raw.energy}
detectors['hsd']={'pskey':'hsd', 'get':lambda det: det.raw.waveforms}
detectors['photonEnergy']={'pskey':'ebeam', 'get':lambda det: det.raw.ebeamPhotonEnergy}

# Important Epics
detectors['vitaraDelay']={'pskey':'las_fs14_target_time', 'get':lambda det: det}

In [ ]:
# Analysis is of form {analysisKey: {'function': analysisFunction(), 'detectorKey': 'key', 'analyzeEvery':1}}
# Function element is optional. If not provided, raw data is returned.
analysis = {}
analysis['vitaraDelay'] = {'function':lambda x: x, 'detectorKey':'vitaraDelay'}
analysis['evrs'] = {'detectorKey':'evrs'}
analysis['vls1D'] = {'function': lambda x: np.sum(x,-1), 'detectorKey':'vls'}
analysis['pulseEnergy'] = {'detectorKey':'gmd'}
analysis['photonEnergy'] = {'detectorKey':'photonEnergy'}
analysis['wfTime'] = {'function': lambda x: x[0]['times'], 'detectorKey':'hsd'}

resample = lambda x, rebin_factor: x.reshape(-1, rebin_factor).mean(1)
analysis['itof-time'] = {'function': lambda x: resample(x[0]['times'],10), 'detectorKey':'hsd'}
analysis['itof-waveform'] = {'function': lambda x: resample(x[0][0],10), 'detectorKey':'hsd'}
analysis['diode-waveform'] = {'function': lambda x: x[9][0].astype(float)[:5000], 'detectorKey':'hsd'}

analysis['atm-proj1'] =  {'function': lambda x: np.sum(x[360:500,:],axis=0), 'detectorKey':'tmo_atmopal'}
analysis['atm-proj2'] =  {'function': lambda x: np.sum(x[0:240,:],axis=0), 'detectorKey':'tmo_atmopal'}
analysis['atm-proj3'] =  {'function': lambda x: np.sum(x[240:360,:],axis=0), 'detectorKey':'tmo_atmopal'}

In [ ]:
import data
import loop
fastData, epicsData = data.XTCReader(exp=exp,
               run=run_number,
               detectors=detectors,
               analysisDict=analysis,
               nread=1,
               loopStyle=lambda itr: loop.timeIt(itr, printEverySec=1))   

# Example plots and analyses
## Histogram of pulse energies

In [ ]:
fastData['atm-proj1'].shape

In [ ]:
plt.hist(fastData['pulseEnergy'] * 1e3, bins=20);
plt.xlabel("Pulse energy / uJ");

## TOF traces

In [ ]:
gas_on = fastData['evrs'][:,70]==1
gas_off = np.logical_not(gas_on)

t = fastData['itof-time'][0,:]
plt.plot(t, fastData['itof-waveform'][gas_on].mean(0), 'k', label='Jet on');
plt.plot(t, fastData['itof-waveform'][gas_off].mean(0), 'r', alpha=0.5, label='Jet off');
plt.title("ion TOF traces");
plt.legend()
plt.xlabel("ToF / us")

# Print detectors available in this (exp, run)
Fast detectors. Make a measurement every shot

In [ ]:
run.detnames

# Print epics detectors available
These are slow detectors. While they write a value for every event, they do not update at 120Hz.

In [ ]:
def getEpics(run):
    epicsNames = []
    for key in run.epicsinfo:
        epicsNames.append( key[0] )
    return epicsNames

In [ ]:
getEpics(run)

# Test data access for a detector
Use a test evt to see what the detector object returns
Many of the detectors call functions are undocumented. Try typing '<Detector Obj>.' then pressing tab to determine the call function. For example,
```python
gmd.[tab]
gmd.raw.[tab]
gmd.raw.energy
help(gmd.raw.energy)
```
shows you that the xray pulse energy may be read by `gmd.raw.energy(evt0)`

In [ ]:
evt0=next( run.events() )

In [ ]:
gmd = run.Detector('gmd') # xray pulse energy monitor

In [ ]:
help(gmd.raw.energy)

In [ ]:
gmd.raw.energy(evt0)